### Inicializando e checando o ecossistema de databases

In [ ]:
#mongodb
!mongo --quiet --eval 'rs.initiate();'
!mongo --quiet --eval 'rs.isMaster().ismaster'

#redis
!redis-server --daemonize yes
!redis-cli ping

#cassandra
!cqlsh --execute="SELECT host_id, bootstrapped FROM system.local;"
!cqlsh --file=./cassandra/initialize.cql
!cqlsh --execute='DESC keyspaces;'

### Inicializando modulos, libs e variaveis

In [ ]:
from pymongo import MongoClient
from pprintpp import pprint
import warnings
import json
from cassandra.cluster import Cluster
from libs.dispute_faker import generate_dispute
import redis

warnings.filterwarnings('ignore')


# Decoder personalizado para Cassandra
def dict_cleaner(items):
    result = {}
    for key, value in items:
        if value is None and key not in ['lastRound', 'counterProposal']:
            value = ''
        if 'gmt' in key:
            try:
                value = str(value*1000)
            except TypeError:
                value = None
        result[key] = value
    return result


mongoclient = MongoClient('localhost', 27017)
dispute_db = mongoclient["disputes"]
print(dispute_db)

cluster = Cluster()
cassandra = cluster.connect('disputes')
pprint(cassandra.execute('DESC tables;').all())

redispute = redis.Redis(host='localhost', port=6379, decode_responses=True)
redispute.ping()

In [ ]:
inserted = dispute_db['disputes'].insert_many([generate_dispute() for _ in range(2000)])
pprint(len(inserted.inserted_ids))

In [ ]:
query = dispute_db['disputes'].find({})
for dispute in query:
    dispute['id'] = str(dispute['_id'])
    dispute['arbitratorId'] = str(dispute['resolution']['arbitratorId']) if not None else ''
    dispute['decision'] = str(dispute['resolution']['decision']) if not None else ''
    dispute['status'] = str(dispute['resolution']['status'])
    dispute['itemId'] = str(dispute['item']['itemId'])
    dispute['skuId'] = str(dispute['item']['skuId'])
    dispute['id'] = str(dispute['_id'])
    dispute.pop('_id')
    dispute = json.dumps(dispute)
    cleaned = json.dumps(json.loads(dispute, object_pairs_hook=dict_cleaner))
    cassandra.execute(f"INSERT INTO disputes JSON'{cleaned}'").all()
pprint(cassandra.execute(f"SELECT COUNT(*) from disputes")[0].count)

In [ ]:
!cqlsh --execute="SELECT arbitratorid, decision, status, disputereason FROM disputes.disputes LIMIT 100;"

In [ ]:
!ls

In [ ]:
!cd /usr/local/etc/cassandra

In [ ]:
!ls